In [18]:
import pandas as pd
import numpy as np

In [33]:
df = pd.read_csv('gender-classifier-DFE-791531.csv', sep=',',encoding='latin-1')

np.random.seed(0)
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]

train.columns
# created
# description
# fav_number
# link_color? 
# name
# profileimage
# sidebar_color
# text
# tweet_count

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'gender', 'gender:confidence', 'profile_yn',
       'profile_yn:confidence', 'created', 'description', 'fav_number',
       'gender_gold', 'link_color', 'name', 'profile_yn_gold', 'profileimage',
       'retweet_count', 'sidebar_color', 'text', 'tweet_coord', 'tweet_count',
       'tweet_created', 'tweet_id', 'tweet_location', 'user_timezone'],
      dtype='object')

# Data Transformations

In [20]:
# DROP NON MALE / FEMALE
genders = list(train.gender)
print(np.unique(genders))
train_drop = train[train.gender.notnull()]
train_drop = train_drop[train_drop.gender != 'brand']
train_drop = train_drop[train_drop.gender != 'unknown']
train_drop.shape
genders = list(train_drop.gender)
train = train_drop
print(np.unique(genders))
print(train_drop.shape)

['brand' 'female' 'male' 'nan' 'unknown']
['female' 'male']
(10320, 26)


In [21]:
# CHANGE ACCOUNT CREATION DATE TO TIME SINCE CREATION
import datetime
def get_length(s):
    year = s.split('/')[2]
    month = s.split('/')[0]
    day = s.split('/')[1]
    return datetime.date(17, 10, 1) - datetime.date(int(year), int(month), int(day))
created = train.created
created = [el.split()[0] for el in created]
c = [get_length(s) for s in created]
months = [el.days/30 for el in c]
train.created = months

In [11]:
texts = list(train.text)
genders = list(train.gender)

In [12]:
print (len(texts), len(genders))

10320 10320


In [5]:
words = {}
for i in range(len(texts)):
    for word in texts[i].split():
        word = word.lower()
        if word not in words:
            words[word] = {'male':0, 'female':0}
        if genders[i] == 'male':
            words[word]['male'] += 1
        elif genders[i] == 'female':
            words[word]['female'] += 1


In [6]:
for key in words.keys():
    words[key]['ratio'] = (words[key]['male']+1)/(words[key]['male']+words[key]['female']+1)
    words[key]['total'] = words[key]['male']+words[key]['female']

In [7]:
biggest_words = [(words[key]['ratio'],key) for key in words.keys() if words[key]['total'] > 30]

In [8]:
sorted(biggest_words)[:20]

[(0.16161616161616163, '#pushawardslizquens'),
 (0.16326530612244897, 'forevermore'),
 (0.18181818181818182, 'bed'),
 (0.18181818181818182, 'mom'),
 (0.1839080459770115, 'everydayiloveyou'),
 (0.1896551724137931, 'literally'),
 (0.21311475409836064, 'family'),
 (0.25, 'hair'),
 (0.2571428571428571, 'room'),
 (0.25862068965517243, '#artistoftheyear'),
 (0.2608695652173913, '_ù÷_'),
 (0.2641509433962264, 'favorite!'),
 (0.2647058823529412, 'together'),
 (0.26973684210526316, 'makes'),
 (0.27906976744186046, 'meet'),
 (0.28, 'girl'),
 (0.28205128205128205, 'sad'),
 (0.2857142857142857, 'fall'),
 (0.29411764705882354, 'movie'),
 (0.29411764705882354, 'voted')]

In [9]:
total = 0
for gender in genders:
    if gender == 'male':
        total += 1
print (total)

4951


In [10]:
total = 0
for gender in genders:
    if gender == 'female':
        total += 1
print (total)

5369


In [11]:
total = 0
for gender in genders:
    if gender == 'brand':
        total += 1
print (total)

4767


In [12]:
fav_numbers = train.fav_number
tweet_created = train.tweet_created
tweet_count = train.tweet_count
created = train.created

In [ ]:
created = train.created
created = [el.split()[0] for el in created]

'12/5/13'

In [14]:
import datetime
def get_length(s):
    year = s.split('/')[2]
    month = s.split('/')[0]
    day = s.split('/')[1]
    return datetime.date(17, 10, 1) - datetime.date(int(year), int(month), int(day))


In [16]:
c = [get_length(s) for s in created]
months = [el.days/30 for el in c]

In [17]:
%matplotlib inline
import matplotlib.pyplot as plt

In [18]:
correlated_pot = np.array([fav_numbers, months, tweet_count])

In [19]:
np.shape(correlated_pot)

(3, 16054)

In [20]:
np.corrcoef(correlated_pot)

array([[ 1.        ,  0.06233283,  0.06626339],
       [ 0.06233283,  1.        ,  0.08133869],
       [ 0.06626339,  0.08133869,  1.        ]])